# qVector

Simulations for qVector idea. 

In [1]:
import projectq
import copy
import numpy as np
import scipy.optimize
from scipy import optimize
from projectq.backends import Simulator
from projectq.ops import (X,
                          Measure,
                          H,
                          C,
                          All,
                          NOT,
                          Swap,
                          get_inverse,
                          BasicMathGate,Rx,Ry,Rz)

## Gate set

All gates are circuit chunks applied to a given quantum register (variable name reg).

In [33]:
def arbitrary_rotation(reg,n,a,b,c):
    """
    reg: a quantum register
    n: target qubit
    a, b, c: angles
    """
    Rz(a) | reg[n]
    Ry(b) | reg[n]
    Rz(c) | reg[n]
    
def get_pta_parameters(t,T1,T2):
    """ Compute standard deviation for
        Pauli twirling simulations
        
        s: array | Standard deviation of the gaussian distributions
        for the x, y and z random rotations
        
        t: float | gate time
        T1: float | qubit decay time 
        T2: float | qubit dephasing time
    """
    s=np.zeros(3)
    M=np.array(3)
    T=np.array([T1,T2,2*T2*T1/(2*T1-T2)])
    M=T/t
    px = (1.-np.exp(-1./M[0]))/4.0
    pz = (1.-np.exp(-1./M[1]))/2.0-px
    s[0] = np.sqrt(-np.log(1.-px))
    s[1] = np.sqrt(-np.log(1.-px))
    s[2] = np.sqrt(-np.log(1.-pz))
    return s

def pauli_twirling(reg,s):
    """
    Set of arbitrary rotations applied to all qubits
        reg: quantum register
        s: set of standard deviations for twirling | floats
    """
    for n in range(0,len(reg)):
        ax =  np.random.normal(0.0,s[0])
        ay =  np.random.normal(0.0,s[1])
        az =  np.random.normal(0.0,s[2])        
        Rx(ax) | reg[n]
        Ry(ay) | reg[n]
        Rz(az) | reg[n]    
        
def xrotation_ladder(reg,p):
    """
        Sequence of X rotations acting on all the qubits
        in the register 
        
        p: array of floats | rotation angles
    """
    if (len(p)!=len(reg)):
        raise Exception("The length of p must be equivalent to the number of qubits")
    for n in range(0,len(reg)):
        Rx(p[n]) | reg[n]
    
def yrotation_ladder(reg,p):
    """
        Sequence of Y rotations acting on all the qubits
        in the register 
        
        p: array of floats | rotation angles
    """
    if (len(p)!=len(reg)):
        raise Exception("The length of p must be equivalent to the number of qubits")
    for n in range(0,len(reg)):
        Ry(p[n]) | reg[n]

def zrotation_ladder(reg,p):
    """
        Sequence of X rotations acting on all the qubits
        in the register 
        
        p: array of floats | rotation angles
    """

    if (len(p)!=len(reg)):
        raise Exception("The length of p must be equivalent to the number of qubits")
    for n in range(0,len(reg)):
        Rz(p[n]) | reg[n]

def arbitrary_noisy_rotations(reg,p,sd):
    """
        Set of arbitrary rotations applied to all qubits
        
        reg: quantum register
        p: set of parameters
        sd: standard deviation
    """
    if (len(p)!=3*len(reg)):
        raise Exception(" Length of p must be 3 times de number of qubits")
    step = len(reg)
    # z rotations
    zrotation_ladder(reg,p[0:step])
    pauli_twirling(reg,sd)
    # y rotations
    yrotation_ladder(reg,p[step:2*step])
    pauli_twirling(reg,sd)
    # z rotations
    zrotation_ladder(reg,p[2*step:3*step])
    pauli_twirling(reg,sd)
        
def even_CNOT_ladder(reg):
    """
        Ladder of CNOT gates applied from all the possible even targets
        (Suitable for a chain architecture)
    """
    for n in range(0,len(reg)-1,2):
        C(NOT) | (reg[n], reg[n+1])
    

def odd_CNOT_ladder(reg):
    """
        Ladder of CNOT gates applied from all the possible odd targets
        (Suitable for a chain architecture)
    """
    for n in range(1,len(reg)-1,2):
        C(NOT) | (reg[n], reg[n+1])

def print_wavefunction(eng):
    """
        Helper function for printing wavefunction on a quantum register
        at a given moment of the execution
    """
    mapping, wavefunction = copy.deepcopy(eng.backend.cheat())
    print("The full wavefunction is: {}".format(wavefunction))
    
def compute_cost_function(eng, reg, n_qubits, n_protected):
    """
        Measure and compute cost function of final wavefunction
    """
    eng.flush()
    prob0 = eng.backend.get_probability('0'*n_protected, reg[0:n_protected])
    Measure | reg
    hamming_distance = 0
    for n in range(0,n_protected):
        hamming_distance += int(reg[n])
    return hamming_distance, prob0
        

## Function for variational circuit 1

In [34]:
# Variational circuit 1

class circuit1():
    """ Class for applying a variational circuits comprising
        two ladders of arbitrary single qubit rotation 
        interleaved with a ladder of CNOTs with odd qubits as controls 
        and another ladder of CNOTs with even qubits as controls

        The circuit motiv is repeated "depth" times and a final ladder of
        arbitrary qubit rotation is applied at the end 
    """
    
    def __init__(self, n_qubits, depth):
        self.n_qubits =  n_qubits
        self.depth = depth
        self.n_parameters = self.get_n_parameters()

    def get_n_parameters(self):
        return 6*self.n_qubits*self.depth + 3*self.n_qubits
    
    def generate_random_p(self):
        p = 4*np.pi*np.random.random(self.get_n_parameters())
        return p

    def __call__(self, eng, reg, p, sq_sd, tq_sd):
        """
        Inputs:
            eng: quantum engine
            reg: quantum register
            depth: integer | number of repetitions of the circuit motiv 
            n_qubits: integer | number of qubits
            p: array of floats with size 
                6*n_qubits*depth + 3*n_qubits| set of parameters
            sq_sd: float | set of PT standard deviations for single qubit gates
            tq_sd: float | set of PT standard deviations for two qubit gates
        """

        if len(p)!= self.n_parameters:
            raise Exception("The number of parameters is inadequate. It should be 6*N*d+3*N")

        #print("Initial")
        #print_wavefunction(eng)

        start = 0
        step = 3*len(reg)
        for epoch in range(0,self.depth):
            arbitrary_noisy_rotations(reg,p[start:start+step],sq_sd)
            start = start + step
            even_CNOT_ladder(reg)
            pauli_twirling(reg,tq_sd)
            arbitrary_noisy_rotations(reg,p[start:start+step],tq_sd)
            start = start + step
            odd_CNOT_ladder(reg)
            pauli_twirling(reg,tq_sd)
            #print("after depth {0}".format(epoch))
            #print_wavefunction(eng)

        arbitrary_noisy_rotations(reg,p[start:start+step],sq_sd)
        #print("Final")
        #print_wavefunction(eng)

    def pure_noise(eng, reg, p, sq_sd, tq_sd):
        """ Function for simulating effect of noise 
            for circuit type 1 without variational circuit

            eng: quantum engine
            reg: quantum register
            p: array of floats with size 
                6*n_qubits*depth + 3*n_qubits| set of parameters
            t_sq: float | single qubit gate time 
            t_tq: float | two qubit gate time
            T1: float | qubit decay time 
            T2: float | qubit dephasing time
        """
        for epoch in range(0,self.depth):
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,tq_sd)
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,sq_sd)
            pauli_twirling(reg,tq_sd)
        pauli_twirling(reg,sq_sd)

In [40]:
# initialize circuit
n_qubits = 3
n_protected = 1
depth = 3
circuit_example = circuit1(n_qubits, depth)
# fix gate times
sqt = 10
tqt = 20
# fix relaxation and dephasing parameters
T1 = 20000
T2 = 2000
# compute standard deviations for PTA
sq_sd = get_pta_parameters(sqt,T1,T2)
tq_sd = get_pta_parameters(tqt,T1,T2)
# generate random p
p = circuit_example.generate_random_p()
# test circuit
eng = projectq.MainEngine(backend=Simulator())
reg  = eng.allocate_qureg(n_qubits)
circuit_example(eng, reg, p, sq_sd, tq_sd)
cost, prob0 = compute_cost_function(eng, reg, n_qubits, n_protected)
print(cost, prob0)

0 0.7592635888289858


## Functions for random state prep


In [41]:
class state_prep_1():
    
    """
    Apply random X gates for state preparation
    The class has functions to perform the state prep
    and the inverse state prep
    """
    
    def __init__(self, n_qubits, n_protected):
        self.n_qubits = n_qubits
        self.n_protected = n_protected
        self.n_parameters = n_protected

    def get_n_parameters(self):
        return self.n_protected
    
    def generate_random_p(self):
        p = np.random.binomial(1,0.5,self.get_n_parameters())
        print(p)
        return p
    
    def state_prep(self, eng, reg, p, sq_sd, tq_sd):
        """ eng: quantum engine
            reg: quantum register
            depth: integer | number of repetitions of the circuit motiv 
            n_qubits: integer | number of qubits
            p: array of floats with size 
                6*n_qubits*depth + 3*n_qubits| set of parameters
            sq_sd: float | set of PT standard deviations for single qubit gates
            tq_sd: float | set of PT standard deviations for two qubit gates
        """

        if len(p)!= self.n_parameters:
            raise Exception("The number of parameters is inadequate. It should be N")

        for n in range(0,self.n_protected):
            if p[n] == 1:
                X | reg[n]
        pauli_twirling(reg,tq_sd)

    def inv_state_prep(self, eng, reg, p, sq_sd, tq_sd):
        """ eng: quantum engine
            reg: quantum register
            depth: integer | number of repetitions of the circuit motiv 
            n_qubits: integer | number of qubits
            p: array of floats with size 
                6*n_qubits*depth + 3*n_qubits| set of parameters
            sq_sd: float | set of PT standard deviations for single qubit gates
            tq_sd: float | set of PT standard deviations for two qubit gates
        """

        if len(p)!= self.n_parameters:
            raise Exception("The number of parameters is inadequate. It should be N")

        for n in range(0,self.n_protected):
            if p[n] == 1:
                X | reg[n]
        pauli_twirling(reg,tq_sd)

    def pure_noise(self, eng, reg, p, sq_sd, tq_sd):
        """ Function for simulating effect of noise 
            for circuit type 1 without variational circuit

            eng: quantum engine
            reg: quantum register
            p: array of floats with size 
                6*n_qubits*depth + 3*n_qubits| set of parameters
            t_sq: float | single qubit gate time 
            t_tq: float | two qubit gate time
            T1: float | qubit decay time 
            T2: float | qubit dephasing time
        """
        pauli_twirling(reg,sq_sd)

In [45]:
# initialize circuit
n_qubits = 3
n_protected = 1
depth = 3
circuit_example = circuit1(n_qubits, depth)
sp_example = state_prep_1(n_qubits, n_protected)
# fix gate times
sqt = 10
tqt = 20
# fix relaxation and dephasing parameters
T1 = 20000
T2 = 2000
# compute standard deviations for PTA
sq_sd = get_pta_parameters(sqt,T1,T2)
tq_sd = get_pta_parameters(tqt,T1,T2)
# generate random p
p_circuit = circuit_example.generate_random_p()
p_sp = sp_example.generate_random_p()
# test circuit
eng = projectq.MainEngine(backend=Simulator())
reg  = eng.allocate_qureg(n_qubits)
sp_example.state_prep(eng, reg, p_sp, sq_sd, tq_sd)
circuit_example(eng, reg, p_circuit, sq_sd, tq_sd)
sp_example.inv_state_prep(eng, reg, p_sp, sq_sd, tq_sd)
cost, prob0 = compute_cost_function(eng, reg, n_qubits, n_protected)
print(cost, prob0)

[1]
0 0.3727639597478133


In [46]:
class autoencoder():
    
    def __init__(self, n_qubits, variational_circuit, varcirc_depth,
                 state_prep_circuit, n_protected, filename, 
                 n_states = 10, repetitions = 1000, 
                 min_angle=0.0, max_angle=4*np.pi, verbose = True):
        """
            n_qubits: integer | number of qubits
            variational_circuit: instance of some variational_circuit_x class 
            varcirc_depth: integer | depth for the variational circuit
            state_prep_circuit: instance of some state_prep_x class
            n_states: integer | number of states to be considered in training
            n_protected: number of qubits protected at the beginning of the circuit
            filename: string | filename for creating output files
            repetitions: integer | number of measurements collected
            min_angle: float | miminum value for rotation angles
            max_angle: float | maximum value for rotation angles
        """
        self.n_qubits = n_qubits
        self.var_circuit = variational_circuit(n_qubits, varcirc_depth)
        self.state_prep =  state_prep_circuit(n_qubits, n_protected)
        self.varcirc_depth = varcirc_depth
        self.n_states = n_states
        self.n_protected = n_protected
        self.filename = filename
        self.verbose=verbose
        self.n_parameters = self.var_circuit.get_n_parameters()
        self.score = 1.0*n_qubits
        self.score_std = 0.0
        self.parameters = np.zeros(self.n_parameters)
        self.n_calls = 0
        self.min = min_angle
        self.max = max_angle
        self.n_repetitions = repetitions
        if self.verbose == True:
            # output file
            output = open(self.filename+'.out','w')
            output.write("INITIALIZING OBJECTIVE:\n")
            output.write("Depth={0}, nqubits={1}, nparameters={2} \n".format(
                self.varcirc_depth,self.n_qubits,self.n_parameters) )
            output.close()
        self.set_of_states = []
        for n in range(0,self.n_states):
            self.set_of_states.append(self.state_prep.generate_random_p())

    def set_noise_parameters(self, t_sq, t_tq, T1, T2):
        self.sq_sd = get_pta_parameters(t_sq,T1,T2)
        self.tq_sd = get_pta_parameters(t_tq,T1,T2)
        
    def __call__(self, parameters):
        
        cost_function_vals = np.zeros(self.n_repetitions*self.n_states)
        probabilities = np.zeros(self.n_repetitions*self.n_states)
        
        counter = -1
        for state in self.set_of_states:
            for n in range(self.n_repetitions):
                
                counter += 1
                eng = projectq.MainEngine(backend=Simulator())
                reg  = eng.allocate_qureg(self.n_qubits)
                
                self.state_prep.state_prep(eng, reg, state, 
                                              self.sq_sd,self.tq_sd)

                self.var_circuit(eng, reg, parameters, 
                                    self.sq_sd,self.tq_sd)

                self.state_prep.inv_state_prep(eng, reg, state, 
                                                  self.sq_sd,self.tq_sd)
                
                cost_function_vals[counter], probabilities[counter] = compute_cost_function(eng, 
                                                                                            reg, 
                                                                                            self.n_qubits,
                                                                                            self.n_protected)
                
        self.parameters = parameters
        self.score = np.log(1.0-np.mean(probabilities))
        self.score_std = np.std(probabilities)
        self.n_calls += 1
        # printing information of the optimization
        if (self.n_calls%5 == 0 and self.verbose == True):
            self.printInfo()
            print(self.n_calls, np.mean(probabilities), self.score, self.score_std)
        
        return self.score

    def printInfo(self):
        output = open(self.filename+'.out','a')
        output.write("Call number: {0}, Score: {1}\n".format(self.n_calls, self.score) )
        print("Current parameters (in units of pi):\n", self.parameters/np.pi, file=output)
        output.write('\n')
        output.close()  
    
    def train(self, optimization_method=None, initial_guess=None):
        """Train the objective using one of the available optimization methods"""
        
        self.n_calls = 0

        if self.verbose == True:
            output = open(self.filename+'.out','a')
            output.write("\nSTARTING OPTIMIZATION WITH OPTIMIZER {}\n".format(optimization_method))
            output.close()
        
        # generating random initial guess
        if initial_guess == None:
            self.initial_guess=4*np.pi*np.random.random(int(self.n_parameters))
        else:
            self.initial_guess = initial_guess
            # initial_guess=np.zeros((self.nrows,self.ncolumns))
        
        if optimization_method == None or optimization_method == 'LBFGS':
            b=[]
            for n in range(0,int(self.n_parameters)):
                b.append((self.min,self.max))

            sol = scipy.optimize.fmin_l_bfgs_b(self, self.initial_guess, args=(), fprime=None,
                                               approx_grad=1, pgtol=1e-04, epsilon=1e-08, factr=1e7,
                                               iprint=0, bounds=b, maxfun=10000, maxiter=5000)
            solution = sol[0]
            
        elif optimization_method == 'COBYLA':
            method = 'COBYLA'
            solution = scipy.optimize.minimize(self, self.initial_guess, args=(), method=method, 
                                               constraints=(), tol=None, callback=None, 
                                               options={'iprint': 0, 'disp': True, 'maxiter': 20000, 
                                                        'tol': 1e-04, 'rhobeg': 0.02}).x
        elif optimization_method == 'POWELL':
            method = 'Powell'
            solution = scipy.optimize.minimize(self, self.initial_guess, args=(), method=method, 
                                               options={'ftol':1e-05,'xtol':1e-04,'maxiter':20000,
                                                        'maxfev':20000}).x
        elif optimization_method == 'ND':
            method = 'Nelder-Mead'
            solution = scipy.optimize.minimize(self, self.initial_guess, args=(), method=method, 
                                               options={'ftol':1e-05,'xtol':1e-04,'maxiter':20000,
                                                        'maxfev':20000}).x
        elif optimization_method == 'basin_hopping':
            minimizer_kwargs = {"method":"L-BFGS-B"}
            bounds = MyBounds([self.xmax]*int(self.n_parameters),[self.xmin]*int(self.n_parameters))
            solution = scipy.optimize.basinhopping(self, self.initial_guess, niter=100, T=0.20, 
                                                   stepsize=0.5, minimizer_kwargs=minimizer_kwargs, 
                                                   take_step=None, accept_test=bounds, callback=None, 
                                                   interval=20, disp=False, niter_success=5).x

        elif optimization_method == 'differential_evolution':
            bounds=[]
            for n in range(0,int(self.n_parameters)):
                bounds.append((self.xmin,self.xmax))
            print(bounds)
            solution = scipy.optimize.differential_evolution(self, bounds, args=(), strategy='best1bin', 
                                                             maxiter=500, popsize=15, tol=0.01, 
                                                             mutation=(0.25, 0.75), recombination=0.9, 
                                                             seed=None, callback=None, disp=False, 
                                                             polish=True, init='latinhypercube').x
        
        self.optimal_parameters = solution
        self.optimal_score = self(solution)
        self.saveinfo(self.filename,optimization_method)
        
        if self.verbose == True:
            output = open(self.filename+'.out','a')
            output.write("OPTIMIZATION COMPLETE SUCESSFULLY\n")
            print("Optimal parameters:\n",self.parameters,file=output)
            output.write('\n')
            output.write("Final score: {0}, Number of calls: {1}\n".format(self.score,self.n_calls))
            output.close()
        
    def saveinfo(self,filename,optimization_method):
        object_file = open(filename+'-'+optimization_method+'.pkl', 'wb')
        pickle.dump(self, object_file)
        object_file.close()

In [55]:
# initialize circuit
n_qubits = 3
depth = 3
n_protected=1
circuit_example = circuit1(n_qubits, depth)
sp_example = state_prep_1(n_qubits, n_protected)
# fix gate times
sqt = 10
tqt = 20
# fix relaxation and dephasing parameters
T1 = 20000
T2 = 2000
# compute standard deviations for PTA
sq_sd = get_pta_parameters(sqt,T1,T2)
tq_sd = get_pta_parameters(tqt,T1,T2)

for m in [10,20,40,80]:
    autoencoder_example = autoencoder(n_qubits, circuit1, depth, state_prep_1, 
                                      n_protected, 'Example_au', 1, m, 0.0, 4*np.pi)

    autoencoder_example.set_noise_parameters(sqt, tqt, T1, T2)

    p = autoencoder_example.var_circuit.generate_random_p()

    autoencoder_example.__call__(p)

[1]
[0]
[0]
[1]


In [57]:
autoencoder_example = autoencoder(n_qubits, circuit1, depth, state_prep_1, 
                                  n_protected, 'Example_au', 2, 50, 0.0, 4*np.pi)

autoencoder_example.set_noise_parameters(sqt, tqt, T1, T2)

autoencoder_example.train(optimization_method=None, initial_guess=None)

[1]
[0]


Exception ignored in: <bound method Qubit.__del__ of <projectq.types._qubit.Qubit object at 0x10e43acc0>>
Traceback (most recent call last):
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/types/_qubit.py", line 127, in __del__
    self.engine.deallocate_qubit(weak_copy)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 151, in deallocate_qubit
    tags=[DirtyQubitTag()] if is_dirty else [])])
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 184, in send
    self.next_engine.receive(command_list)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_tagremover.py", line 56, in receive
    self.send([cmd])
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 184, in send
    self.next_engine.receive(command_list)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_optimize.py", line 237, in receive
    self._cache_cm

5 0.806177260311 -1.64081125055 0.1123759144
10 0.811823902342 -1.6703770649 0.113331536773
15 0.815471321524 -1.68995038869 0.10921001908
20 0.808201658216 -1.65131076212 0.1130666983
25 0.815184574492 -1.6883976519 0.113309811569
30 0.813574829845 -1.67972535283 0.116047559888
35 0.803670315508 -1.6279599691 0.113688703395
40 0.810155495065 -1.66154993697 0.115740731911


KeyboardInterrupt: 

In [62]:
# initialize circuit
n_qubits = 5
depth = 2
n_protected=1
circuit_example = circuit1(n_qubits, depth)
sp_example = state_prep_1(n_qubits, n_protected)
# fix gate times
sqt = 10
tqt = 20
# fix relaxation and dephasing parameters
T1 = 20000
T2 = 2000
# compute standard deviations for PTA
sq_sd = get_pta_parameters(sqt,T1,T2)
tq_sd = get_pta_parameters(tqt,T1,T2)

autoencoder_example = autoencoder(n_qubits, circuit1, depth, state_prep_1, 
                                  n_protected, 'Example_au', 2, 50, 0.0, 4*np.pi)

autoencoder_example.set_noise_parameters(sqt, tqt, T1, T2)

autoencoder_example.train(optimization_method=None, initial_guess=None)

[0]
[1]
5 0.45583446593 -0.608501787843 0.0347248392255


Exception ignored in: <bound method Qubit.__del__ of <projectq.types._qubit.Qubit object at 0x10eaa9630>>
Traceback (most recent call last):
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/types/_qubit.py", line 127, in __del__
    self.engine.deallocate_qubit(weak_copy)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 151, in deallocate_qubit
    tags=[DirtyQubitTag()] if is_dirty else [])])
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 184, in send
    self.next_engine.receive(command_list)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_tagremover.py", line 56, in receive
    self.send([cmd])
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_basics.py", line 184, in send
    self.next_engine.receive(command_list)
  File "/Users/jrf/.local/lib/python3.5/site-packages/projectq/cengines/_optimize.py", line 237, in receive
    self._cache_cm

Invalid qubit pipeline encountered (in the process of shutting down?).
Invalid qubit pipeline encountered (in the process of shutting down?).
10 0.45496411484 -0.606903642156 0.0356838285897
15 0.455747612304 -0.608342191746 0.0342909781356
20 0.456737572705 -0.610162784276 0.0347122182797
25 0.451187982673 -0.599999305349 0.0391175865676
30 0.453071601037 -0.60343738282 0.0325628395401
35 0.452435502729 -0.602275020995 0.03730339833
40 0.456060627816 -0.608917486529 0.0388182988527
45 0.455411461736 -0.607724745119 0.0385341909953
50 0.458560525311 -0.613523992216 0.0389712750614
55 0.455356118773 -0.607623126839 0.0410290672576
60 0.455758395902 -0.608362005541 0.0412747454992
65 0.456489119583 -0.609705553456 0.0314040411374
70 0.45603513253 -0.608870616069 0.0331920566548
75 0.454123461221 -0.605362448258 0.0306997719069
80 0.998020502217 -6.22491211135 0.0012449555741
85 0.997859189845 -6.14657094439 0.00141352781582
90 0.998116804859 -6.27478540155 0.00111696484777
95 0.998063254

KeyboardInterrupt: 

## Pending
### Improve time by activating parallel computation

### Multiple scenerios to consider in the simulation.

#### We start with an N-qubit register and we allow, where the protected register is only N-k. We allow errors to affect the entire register during the state preparation and during the inverse of the state preparation, even when this is performed only on the protected register
#### We allow the errors to accumulate in the entire register but reset the syndrome register to zeros before we apply the variational circuit and measure the syndrome measurement afterwards.
#### We can also assume that we isolate the protected register from the syndrome register during state preparation and the inverse state preparation, such as the two registers only interact during the evolution of the variational circuit.

### Try example with random rotation circuit as preparation for a 1-qubit protected register.